In [12]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Product elements
product_elements = ['power supply',
                    'motor',
                    'heating unit',
                    'fan',
                    'control system',
                    'casing']

# Design structure Matrix (DSM)
design_structure_matrix =  [[0,1,1,0,0,0],
                            [1,0,0,1,1,1],
                            [1,0,0,1,1,1],
                            [1,1,1,0,0,1],
                            [0,0,1,1,0,1],
                            [1,1,1,1,1,0]]

# Direct likelihood matrix (l)
direct_likelihood_matrix = [[0.0,0.3,0.3,0.0,0.0,0.0],
                            [0.9,0.0,0.0,0.6,0.3,0.6],
                            [0.9,0.0,0.0,0.6,0.3,0.6],
                            [0.3,0.6,0.9,0.0,0.0,0.9],
                            [0.0,0.0,0.3,0.6,0.0,0.3],
                            [0.3,0.9,0.6,0.9,0.6,0.0]]

# Direct impact matrix (i)
direct_impact_matrix =     [[0.0,0.9,0.9,0.0,0.0,0.0],
                            [0.9,0.0,0.0,0.6,0.3,0.3],
                            [0.6,0.0,0.0,0.3,0.3,0.3],
                            [0.3,0.3,0.6,0.0,0.0,0.3],
                            [0.0,0.0,0.3,0.3,0.0,0.3],
                            [0.3,0.6,0.6,0.9,0.6,0.0]]

In [37]:
A = pd.DataFrame(design_structure_matrix, index=product_elements, columns=product_elements)
G = nx.from_pandas_adjacency(A, create_using=nx.DiGraph)

In [3]:
# Direct risk matrix (r)
def direct_risk_matrix(direct_likelihood_matrix, direct_impact_matrix):
    return (np.array(direct_likelihood_matrix)*np.array(direct_impact_matrix)).tolist()

drm = direct_risk_matrix(direct_likelihood_matrix, direct_impact_matrix)

In [10]:
def propagation_tree(graph,target,source):

    propagation_tree = nx.DiGraph()

    for path in list(nx.all_simple_paths(graph, source, target)):
        new_path = []
        for level, node in enumerate(path, 1):
            node_id = ""
            for i in range(level):
                if i == 0:
                    node_id = str(path[i])
                else:
                    node_id = node_id + "-" + str(path[i])
            node_name = node
            node_level = level
            new_path.append(node_id)
            propagation_tree.add_node(node_id, name=node_name, level=node_level)
        nx.add_path(propagation_tree, new_path)
    
    return propagation_tree

def plot_propagation_tree(propagation_tree):
        labels = nx.get_node_attributes(propagation_tree, 'name') 
        pos = hierarchy_pos(propagation_tree)
        plt.figure(figsize=(40,10))
        options = {"with_labels": True, 
                "node_shape": "s", 
                "arrows": False,
                "node_color": "white", 
                "edgecolors": "black", 
                "node_size": 1500}
        nx.draw_networkx(propagation_tree, 
                        pos=pos, 
                        labels=labels, 
                        **options)

def hierarchy_pos(G, root=None, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5):

    '''
    From Joel's answer at https://stackoverflow.com/a/29597209/2966723.  
    Licensed under Creative Commons Attribution-Share Alike 
    
    If the graph is a tree this will return the positions to plot this in a 
    hierarchical layout.
    
    G: the graph (must be a tree)
    
    root: the root node of current branch 
    - if the tree is directed and this is not given, 
        the root will be found and used
    - if the tree is directed and this is given, then 
        the positions will be just for the descendants of this node.
    - if the tree is undirected and not given, 
        then a random choice will be used.
    
    width: horizontal space allocated for this branch - avoids overlap with other branches
    
    vert_gap: gap between levels of hierarchy
    
    vert_loc: vertical location of root
    
    xcenter: horizontal location of root
    '''
    if not nx.is_tree(G):
        raise TypeError('cannot use hierarchy_pos on a graph that is not a tree')

    if root is None:
        if isinstance(G, nx.DiGraph):
            root = next(iter(nx.topological_sort(G)))  #allows back compatibility with nx version 1.11
        else:
            root = random.choice(list(G.nodes))

    def _hierarchy_pos(G, root, width=1., vert_gap = 0.2, vert_loc = 0, xcenter = 0.5, pos = None, parent = None):
        '''
        see hierarchy_pos docstring for most arguments

        pos: a dict saying where all nodes go if they have been assigned
        parent: parent of this branch. - only affects it if non-directed

        '''
    
        if pos is None:
            pos = {root:(xcenter,vert_loc)}
        else:
            pos[root] = (xcenter, vert_loc)
        children = list(G.neighbors(root))
        if not isinstance(G, nx.DiGraph) and parent is not None:
            children.remove(parent)  
        if len(children)!=0:
            dx = width/len(children) 
            nextx = xcenter - width/2 - dx/2
            for child in children:
                nextx += dx
                pos = _hierarchy_pos(G,child, width = dx, vert_gap = vert_gap, 
                                    vert_loc = vert_loc-vert_gap, xcenter=nextx,
                                    pos=pos, parent = root)
        return pos

            
    return _hierarchy_pos(G, root, width, vert_gap, vert_loc, xcenter)

In [45]:
# Combined likelihood matrix (L)
def combined_likelihood_matrix(design_structure_matrix,direct_likelihood_matrix):
    '''Returns the Combined likelihood matrix (L)'''
    number_elements = len(design_structure_matrix)
    g = nx.from_numpy_matrix(np.transpose(np.matrix(design_structure_matrix)), create_using=nx.DiGraph)
    combined_likelihood_matrix = [[0 for col in range(number_elements)] for row in range(number_elements)]
    for target, row in enumerate(design_structure_matrix):
        for source, element in enumerate(row):
            if target != source:
                #print(f"Change source: {source} Change target: {target}")
                # DiGraph representing the propagation tree
                tree = propagation_tree(g,target,source)
                #plot_propagation_tree(tree)
                # List of paths from longest to shortest
                nodes_list = list(reversed(sorted(tree.nodes, key=len)))
                # For each node (representing path in the tree)
                for node in nodes_list:
                    #print(f"Node: {node}  Data: {tree.nodes[node]}")
                    # List of parents, i.e. paths (should be only one, except for the root) that lead to the current path (node) 
                    parents = list(tree.predecessors(node))
                    #print(f"Parents: {parents}")
                    # List of children, i.e. paths that are descendants of the current path (node)
                    children = list(tree.successors(node))
                    #print(f"Children: {children}")
                    if parents != []:
                        parent = parents[0]
                        #print(f"Node: {node}  Data: {tree.nodes[node]} Parent: {parent} Children: {children}")
                        # If the node is a leaf
                        if len(children) == 0:
                            tree.edges[parent,node]['likelihood'] = direct_likelihood_matrix[tree.nodes[node]['name']][tree.nodes[parent]['name']]
                        elif len(children) == 1:
                            child = children[0]
                            tree.edges[parent,node]['likelihood'] = tree.edges[node,child]['likelihood']
                        else:
                            temp = 1
                            for child in children:
                                temp = temp * (1 - tree.edges[node,child]['likelihood'])
                            tree.edges[parent,node]['likelihood'] = 1 - temp
                        #print(f"Likelihood: {tree.edges[parent,node]['likelihood']}")
                    else:
                        #print(f"Node: {node}  Data: {tree.nodes[node]} Children: {children}")
                        #print(f"Root node reached")
                        temp = 1
                        for child in children:
                            temp = temp * (1 - tree.edges[node,child]['likelihood'])
                        likelihood = 1 - temp
                        #print(f"Likelihood_{source},{target}: {likelihood}")
                        combined_likelihood_matrix[target][source] = likelihood
    return combined_likelihood_matrix

clm = combined_likelihood_matrix(design_structure_matrix,direct_likelihood_matrix)
clm

[[0,
  0.9717524751000001,
  0.9903110989593,
  0.9988601104814627,
  0.9966767069430399,
  0.9966767069430399],
 [0.9999999999184375,
  0,
  0.9999999322978819,
  0.9999999992289755,
  0.9999999882559488,
  0.9999999518109696],
 [0.9999999924173628,
  0.9999993955168026,
  0,
  0.9999999960662016,
  0.9999999882559488,
  0.99999990165504],
 [0.9999999999999541,
  0.9999999998628,
  0.9999999999724634,
  0,
  1.0,
  0.999999982789632],
 [0.9999999414322212,
  0.9999995731211457,
  0.9999925950808326,
  0.99612443958372,
  0,
  0.99986507071488],
 [0.9999999999999997,
  0.9999999985613537,
  0.9999999999999755,
  0.9999999997246342,
  0.9999999999997541,
  0]]